In [5]:
def dummy_func1(x, workflow_id=None):
    return x * 2


def dummy_func2(y, workflow_id=None):
    return y + y


def dummy_func3(z, w, workflow_id=None):
    return {"r": z + w}


In [1]:
def setup_local_dask_cluster():
    from dask.distributed import Client, LocalCluster
    from flowcept.flowceptor.plugins.dask.dask_plugins import (
        FlowceptDaskSchedulerPlugin,
        FlowceptDaskWorkerPlugin,
    )

    cluster = LocalCluster(n_workers=2)
    scheduler = cluster.scheduler
    client = Client(scheduler.address)

    # Instantiate and Register FlowceptPlugins
    scheduler_plugin = FlowceptDaskSchedulerPlugin(scheduler)
    scheduler.add_plugin(scheduler_plugin)

    worker_plugin = FlowceptDaskWorkerPlugin()
    client.register_worker_plugin(worker_plugin)

    return client

## Initializing Dask Cluster

In [2]:
dask_client = setup_local_dask_cluster()
dask_client

/Users/rsr/opt/miniconda3/envs/flowcept/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56040 instead
  warnings.warn(


## Initializing Flowcept Consumer

In [3]:
from flowcept.flowcept_api import FlowceptConsumerAPI

In [4]:
consumer = FlowceptConsumerAPI()
consumer.start()

Flowcept Observer starting...
Ok, we're observing!


## Running a simple workflow

In [7]:
import numpy as np
from uuid import uuid4
i1 = np.random.random()
i1 = np.random.random()
wf_id = f"wf_{uuid4()}"
print(f"Workflow_Id={wf_id}")
o1 = dask_client.submit(dummy_func1, i1, workflow_id=wf_id)
o2 = dask_client.submit(dummy_func2, o1, workflow_id=wf_id)
o3 = dask_client.submit(dummy_func3, o1, o2, workflow_id=wf_id)
print(f"Result={o3.result()}")
print(f"Task3_id={o3.key}")

Workflow_Id=wf_523f0356-279f-4bf4-a0fd-7099e888f6ce
Going to send to Redis an intercepted message:
	{"task_id": "dummy_func1-5c9abf960cdbe419c9c1bdc7d44b25ac", "custom_metadata": {"scheduler": "tcp://127.0.0.1:56041", "scheduler_id": "Scheduler-fcbdb35d-e02e-4625-a763-ef85d7f63170", "scheduler_pid": 54082}, "status": "SUBMITTED", "utc_timestamp": 1675750224.341255, "activity_id": "dummy_func1", "used": {"arg0": 0.25180804780289245}, "workflow_id": "wf_523f0356-279f-4bf4-a0fd-7099e888f6ce"}
An intercepted message was received.
Going to send to Redis an intercepted message:
	{"task_id": "dummy_func2-3dbfdcc8b109576f7a3189a1c3fd7f35", "custom_metadata": {"scheduler": "tcp://127.0.0.1:56041", "scheduler_id": "Scheduler-fcbdb35d-e02e-4625-a763-ef85d7f63170", "scheduler_pid": 54082}, "status": "SUBMITTED", "utc_timestamp": 1675750224.356819, "dependencies": ["dummy_func1-5c9abf960cdbe419c9c1bdc7d44b25ac"], "activity_id": "dummy_func2", "used": {"arg0": "dummy_func1-5c9abf960cdbe419c9c1bdc7d4

In [12]:
from flowcept.flowcept_api import TaskQueryAPI
query_api = TaskQueryAPI()

## Retrieving all tasks from this workflow

In [11]:
_filter = {"workflow_id": wf_id}
query_api.query(_filter)

[{'task_id': 'dummy_func1-5c9abf960cdbe419c9c1bdc7d44b25ac',
  'custom_metadata': {'scheduler': 'tcp://127.0.0.1:56041',
   'scheduler_id': 'Scheduler-fcbdb35d-e02e-4625-a763-ef85d7f63170',
   'scheduler_pid': 54082},
  'status': 'FINISHED',
  'utc_timestamp': 1675750224.341255,
  'activity_id': 'dummy_func1',
  'used': {'arg0': 0.25180804780289245},
  'workflow_id': 'wf_523f0356-279f-4bf4-a0fd-7099e888f6ce',
  'debug': True,
  'address': 'tcp://127.0.0.1:56048',
  'generated': {'arg1': 0.5036160956057849},
  'start_time': 1675732224.3790236,
  'end_time': 1675732224.3790295},
 {'task_id': 'dummy_func2-3dbfdcc8b109576f7a3189a1c3fd7f35',
  'custom_metadata': {'scheduler': 'tcp://127.0.0.1:56041',
   'scheduler_id': 'Scheduler-fcbdb35d-e02e-4625-a763-ef85d7f63170',
   'scheduler_pid': 54082},
  'status': 'FINISHED',
  'utc_timestamp': 1675750224.356819,
  'dependencies': ['dummy_func1-5c9abf960cdbe419c9c1bdc7d44b25ac'],
  'activity_id': 'dummy_func2',
  'used': {'arg0': 'dummy_func1-5c9a

## Retrieving only task3

In [15]:
_filter = {"task_id": o3.key}
query_api.query(_filter)

[{'task_id': 'dummy_func3-e4c414c83c7721f9e9e221634b04df50',
  'custom_metadata': {'scheduler': 'tcp://127.0.0.1:56041',
   'scheduler_id': 'Scheduler-fcbdb35d-e02e-4625-a763-ef85d7f63170',
   'scheduler_pid': 54082},
  'status': 'FINISHED',
  'utc_timestamp': 1675750224.361974,
  'dependencies': ['dummy_func2-3dbfdcc8b109576f7a3189a1c3fd7f35',
   'dummy_func1-5c9abf960cdbe419c9c1bdc7d44b25ac'],
  'activity_id': 'dummy_func3',
  'used': {'arg0': 'dummy_func1-5c9abf960cdbe419c9c1bdc7d44b25ac',
   'arg1': 'dummy_func2-3dbfdcc8b109576f7a3189a1c3fd7f35'},
  'workflow_id': 'wf_523f0356-279f-4bf4-a0fd-7099e888f6ce',
  'debug': True,
  'address': 'tcp://127.0.0.1:56048',
  'generated': {'r': 1.5108482868173547},
  'start_time': 1675732224.3884666,
  'end_time': 1675732224.3884807}]